ModuleNotFoundError: No module named 'paths'

In [3]:
import sys
sys.path.insert(0, '../src')

In [4]:
import pandas as pd
from paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR/'tabular_data.parquet')

df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-01-29,1,0.0
1,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2022-01-30,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,2022-01-31,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,1.0,0.0,1.0,1.0,0.0,0.0,2022-02-01,1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-02-02,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73135,3.0,4.0,1.0,1.0,1.0,0.0,1.0,2.0,2.0,3.0,...,29.0,15.0,4.0,12.0,7.0,11.0,4.0,2022-10-27,265,12.0
73136,9.0,4.0,3.0,3.0,2.0,4.0,2.0,6.0,1.0,3.0,...,29.0,17.0,12.0,4.0,5.0,8.0,9.0,2022-10-28,265,3.0
73137,7.0,6.0,3.0,4.0,3.0,5.0,7.0,6.0,5.0,10.0,...,29.0,13.0,9.0,10.0,5.0,10.0,7.0,2022-10-29,265,6.0
73138,6.0,5.0,8.0,6.0,6.0,0.0,1.0,2.0,8.0,6.0,...,8.0,10.0,7.0,3.0,3.0,6.0,2.0,2022-10-30,265,10.0


In [7]:
from datetime import datetime
from data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df, 
    cutoff_date = datetime(2022, 7,1,0,0,0),
    target_column_name = 'target_rides_next_hour'
)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((40545, 674), (40545,), (32595, 674), (32595,))

In [11]:
import numpy as np
class BaselineModelPreviousHour:
    """
    Prediction = actual demand observered in last hour
    """
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame):
        return X_test[f'rides_previous_1_hour']


In [12]:
model = BaselineModelPreviousHour()
predictions = model.predict(X_test)
predictions

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
32590    4.0
32591    9.0
32592    7.0
32593    2.0
32594    4.0
Name: rides_previous_1_hour, Length: 32595, dtype: float32

In [13]:
%reload_ext autoreload
%autoreload 2

In [15]:
pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 394.3 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 388.1 kB/s eta 0:00:0000:0100:11
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 289.3 kB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [16]:
from sklearn.metrics import mean_absolute_error

test_mae = mean_absolute_error(y_test,predictions)
print(f'{test_mae =:.4f}')

test_mae =5.9029


In [17]:
import pandas as pd
import numpy as np

class BaselineModelPreviousWeek:
    """
    Prediction = actual demand observed at t-7 days
    """

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_train: pd.DataFrame):
        return X_test[f'rides_previous_{7*24}_hour']
        

In [18]:
model1 = BaselineModelPreviousWeek()

prediction1= model1.predict(X_test)
test_mae1 = mean_absolute_error(y_test, prediction1)

print(f'{test_mae1 =:.4f}')



test_mae1 =3.3396


In [21]:
class BaselineModelLast4Weeks:
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame):
        return 0.25*(
            X_test[f'rides_previous_{7*24}_hour']+ \
            X_test[f'rides_previous_{2*7*24}_hour']+ \
            X_test[f'rides_previous_{3*7*24}_hour']+ \
            X_test[f'rides_previous_{4*7*24}_hour']
        )

In [22]:
model2 = BaselineModelLast4Weeks()

prediction2= model2.predict(X_test)
test_mae2 = mean_absolute_error(y_test, prediction2)

print(f'{test_mae2 =:.4f}')

test_mae2 =2.9005
